## Sample data investigation notebook
aka the *Decimal Project*

This notebook is the next step after establishing a Hadoop cluster per this [tutorial] (https://github.com/Archethought/tutorials/blob/master/openstack-decimal-proj/containerized_Hadoop.md)


In [1]:
# Basic setup required in all notebooks

# Explicitly define python 2 since we have both 2 & 3 installed
import os
os.environ['PYSPARK_PYTHON'] = '/opt/conda/envs/python2/bin/python'

# Establish spark context or session
import pyspark
sc = pyspark.SparkContext('local[*]') #local spark-master

# Establish SparkSQL context so we can use dataframes
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

# Snakebite is a python library for HDFS access
# See https://snakebite.readthedocs.io/en/latest/
from snakebite.client import Client


# do something to prove it works
rdd = sc.parallelize(range(1000))
rdd.takeSample(False, 5)

[975, 920, 6, 819, 713]

In [2]:
# Handy methods

def HdfsFileToRDD(pDatafile):
    """

    Args:

    Returns:
        
    """
    return sc.parallelize(client.text([pDatafile]))


def prSomeRDD(inputRDD,howMany,msg=None):
    """

    Args:

    Returns:
        
    """
    print msg
    for x in inputRDD.take(howMany):
        print x    
#take(): inputRDD.take(100).foreach(println)

In [3]:
# Define HDFS access 
client = Client("172.18.0.4", 9000 , use_trash=False, effective_user="root")
#help(client)

In [5]:
# Define the datafile and establish access

datafileHDFS = r'/user/root/testData/fotd.csv'
dataRDD = HdfsFileToRDD(datafileHDFS)

In [4]:
stop here

SyntaxError: invalid syntax (<ipython-input-4-f9ef205a8f26>, line 1)

In [ ]:
# dataGEN is a generator with 1 item: the entire file. Only the first access returns data.
# What can we do with the file handle?
dataGEN = client.text([datafile])
print type(dataGEN)
help(dataGEN)

In [ ]:
def CatNGenerator(g):
    i = 0
    for line in g:
        print 'line {0}: {1} '.format(i,line)
        i =+1

def GeneratorToList(g):
    return_list =[]
    for line in g:
        return_list.append(line)
    return return_list

#CatNGenerator(dataGEN)
dataList = GeneratorToList(dataGEN)

In [ ]:
print len(dataList)
print dataList

In [11]:
#dataRDD = (sc.parallelize(client.text([pDatafile]))    .

print "dataRDD: ",type(dataRDD), dataRDD.getNumPartitions(),dataRDD.count(), dataRDD
print dataRDD
dataRDD.take(1) #works
#prSomeRDD(dataRDD,1) #loop thru elements - works
#help(dataRDD)

dataRDD:  <class 'pyspark.rdd.RDD'> 16 1 ParallelCollectionRDD[3] at parallelize at PythonRDD.scala:423
ParallelCollectionRDD[3] at parallelize at PythonRDD.scala:423


['sheet,Date,Event,Lesson,attendees_reg,attendees_student,attendees_special,attendees_board,addendees_free,addendees_coupon,addendees_comp,fee_reg,fee_student,fee_special,fee_board,donations,caller,band\r\n4/16/2016,4/16/2016,Contra Dance Financial Worksheet,-1,37,11,0,4,6,0,6,$9.00,$5.00,$8.00,$9.00,$37.00,Patricia Dancen,Steam\r\n4/2/2016,4/2/2016,Contra Dance Financial Worksheet,0,41,6,0,3,2,2,5,$9.00,$5.00,$8.00,$9.00,$61.00,Duffy Boyle,Balance & Swing\r\n3/19/2016,3/19/2016,Contra Dance Financial Worksheet,0,56,6,1,4,0,5,5,$9.00,$5.00,$8.00,$9.00,$78.00,Sam Smith,John Reading/Josey Toney\r\n3-5-16  Special,3/5/2016,Contra Dance Financial Worksheet,0,132,10,1,4,0,2,10,$9.00,$5.00,$8.00,$9.00,$81.00,Paul Somlo,Nova\r\n2/20/2016,2/20/2016,Contra Dance Financial Worksheet,0,49,8,1,3,0,5,9,$9.00,$5.00,$8.00,$9.00,$41.00,Rick Smith,Prairie Dog Picnic\r\n2/6/2016,2/6/2016,Contra Dance Financial Worksheet,0,52,9,1,3,2,3,3,$9.00,$5.00,$8.00,$9.00,$21.00,Ed Hall,Purple Zephyr\r\n1/16/2016,1

In [13]:
dataRDD2 = dataRDD.map(lambda x: x.split('\r\n'))
print type(dataRDD2), dataRDD2
print dataRDD2.count() 
prSomeRDD(dataRDD2,1) 

<class 'pyspark.rdd.PipelinedRDD'> PythonRDD[23] at RDD at PythonRDD.scala:43
1
None
['sheet,Date,Event,Lesson,attendees_reg,attendees_student,attendees_special,attendees_board,addendees_free,addendees_coupon,addendees_comp,fee_reg,fee_student,fee_special,fee_board,donations,caller,band', '4/16/2016,4/16/2016,Contra Dance Financial Worksheet,-1,37,11,0,4,6,0,6,$9.00,$5.00,$8.00,$9.00,$37.00,Patricia Dancen,Steam', '4/2/2016,4/2/2016,Contra Dance Financial Worksheet,0,41,6,0,3,2,2,5,$9.00,$5.00,$8.00,$9.00,$61.00,Duffy Boyle,Balance & Swing', '3/19/2016,3/19/2016,Contra Dance Financial Worksheet,0,56,6,1,4,0,5,5,$9.00,$5.00,$8.00,$9.00,$78.00,Sam Smith,John Reading/Josey Toney', '3-5-16  Special,3/5/2016,Contra Dance Financial Worksheet,0,132,10,1,4,0,2,10,$9.00,$5.00,$8.00,$9.00,$81.00,Paul Somlo,Nova', '2/20/2016,2/20/2016,Contra Dance Financial Worksheet,0,49,8,1,3,0,5,9,$9.00,$5.00,$8.00,$9.00,$41.00,Rick Smith,Prairie Dog Picnic', '2/6/2016,2/6/2016,Contra Dance Financial Worksheet

In [ ]:
def set_hadoop_config(creds):

    #Required parameter names listed in http://spark.apache.org/docs/latest/storage-openstack-swift.html

    prefix = "fs.hdfs.service." + 'archethought'  #some refs use creds['name'], which can be any string
    hconf = sc._jsc.hadoopConfiguration()
    hconf.set(prefix + ".auth.url", r'http://172.18.0.4' )
    hconf.set(prefix + ".username", 'root')
    hconf.set(prefix + ".http.port", '9000')
    hconf.setBoolean(prefix + ".public", True)

    #hconf.set("fs.hdfs.impl", org.apache.hadoop.hdfs.DistributedFileSystem.getName());
    #hconf.set("fs.file.impl", org.apache.hadoop.fs.LocalFileSystem.getName());
    #hconf.set("fs.swift.impl", "org.apache.hadoop.fs.swift.snative.SwiftNativeFileSystem")


In [ ]:
# spark 2
datafileHDFS = r'hdfs://user/root/testData/fotd.csv'
d = sc.textFile(datafileHDFS)
print type(d)

In [ ]:
d.take(1).foreach(println)